In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import quandl

In [3]:
#function to retrieve tickers info from a file
def get_tickers(path):
    companies = pd.read_csv(path)
    tickers = list(companies['Ticker'])
    print(f"Retrieved {len(tickers)} ticker symbols.")
    return tickers

In [4]:
#function to retrieve prices from ticker
def get_prices(ticker):
    print(f"Retrieving data for {ticker}")
    prices = quandl.get(f"WIKI/{ticker}")['Adj. Close'].reset_index()
    prices['Ticker'] = ticker
    return prices

In [35]:
data = []
ticker_list = get_tickers('data/companies.csv')

Retrieved 30 ticker symbols.


In [36]:
for ticker in ticker_list:
    prices = get_prices(ticker)
    data.append(prices)

Retrieving data for AAPL
Retrieving data for NFLX
Retrieving data for AMZN


LimitExceededError: (Status 429) (Quandl Error QELx01) You have exceeded the anonymous user limit of 50 calls per day. To make more calls today, please register for a free Quandl account and then include your API key with your requests.

In [37]:
def acquire():
    data = []
    ticker_list = get_tickers('data/companies.csv')
    for ticker in ticker_list:
        prices = get_prices(ticker)
        data.append(prices)
    return data

In [17]:
data[0]

,Date,Adj. Close,Ticker
0,1980-12-12,0.422706,AAPL
1,1980-12-15,0.400652,AAPL
2,1980-12-16,0.371246,AAPL
3,1980-12-17,0.380362,AAPL
4,1980-12-18,0.391536,AAPL
...,...,...,...
9395,2018-03-21,171.270000,AAPL
9396,2018-03-22,168.845000,AAPL
9397,2018-03-23,164.940000,AAPL
9398,2018-03-26,172.770000,AAPL


In [18]:
data[1]

,Date,Adj. Close,Ticker
0,2002-05-23,1.196429,NFLX
1,2002-05-24,1.210000,NFLX
2,2002-05-28,1.157143,NFLX
3,2002-05-29,1.103571,NFLX
4,2002-05-30,1.071429,NFLX
...,...,...,...
3983,2018-03-21,316.480000,NFLX
3984,2018-03-22,306.700000,NFLX
3985,2018-03-23,300.940000,NFLX
3986,2018-03-26,320.350000,NFLX


In [19]:
#function to concat the dataframe and pivot it so that each row is a date, each column is a company and the value
#in pivot are the the company's stock prices
def concat_pivot(data, rows, columns, values):
    df = pd.concat(data, sort=True)
    return df.pivot_table(values=values, columns=columns, index=rows)

In [20]:
def compute_returns(df):
    return df.pct_change()

In [21]:
def risk_ratio(df, days=30):
    means = pd.DataFrame(df.tail(days).mean)
    std = pd.DataFrame(df.tail(days).std)
    ratios = pd.concat([means, std], axis=1).reset_index()
    ratios.columns = ['Company', 'Mean', 'StD']
    ratios['Ratio'] = ratios['Mean']/ratios['StD']
    return ratios

In [23]:
#get just the top 10 companies by ratio
#top10 = ratios.sort_values('Ratio', ascending=False).head(10)

In [24]:
def corr(df, days=30):
    return df.tail(days).corr

#target_list = returns[list(top10['Company'])]
#correlation = corr(target_list)

In [26]:
def barchart(df, x, y, length=8, width=16, title=''):
    df = df.sort_values(x, ascending=False)
    plt.figure(figsize=(width, length))
    chart = sns.barplot(data=df, x=x, y=y)
    plt.title(f'{title}\n', fontsize=16)
    return chart

In [27]:
#bar_plot = barchart(top10, 'Ratio', 'Company', title='Stock Return vs. Risk Ratios')

In [29]:
def correlation_plot(corr, title=''):
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    plt.subplots(figsize=(15, 10))
    cmap = sns.diverging_palette(6, 255, as_cmap=True)
    chart = sns.heatmap(corr, mask=mask, cmap=cmap, center=0, linewidths=.5, annot=True, fmt='.2f')
    plt.title(title, fontsize=16)
    return chart

In [31]:
#corr_plot = correlation_plot(correlation, title='Stock Return Correlation')

In [32]:
#function to save as file
def export_file(chart, title):
    fig = chart.get_figure()
    fig.savefig(f'{title}.png')